<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/masked_language_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT for masked language modelling


In [1]:
!pip3 -q install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00


In [2]:
import transformers
import datasets
import torch

# BERT: masked language modelling output

* But BERT is trained to predict masked words, let's try!

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased") #you can also use the trusty "TurkuNLP/bert-base-finnish-cased-v1"
bert = transformers.AutoModelForPreTraining.from_pretrained("bert-base-cased")

In [4]:
# Tell the model it is not really being trained (disables dropout for example)
# I do not think this is needed but am playing it safe, the docs say it is put to eval mode upon load: https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained.config
bert = bert.eval()

...now let's see how well this works for the masked word prediction...
* we need to find the most likely predicted words
* which can be achieved by arg-sorting the predictions and picking top N words
* this is easy and we have done this kind of stuff before
* now let's try straight in torch without a roundtrip to numpy

In [7]:
texts = ["Dogs like to [MASK] cats. They are cute."]

t = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
bert_out = bert(**t)
print(bert_out["prediction_logits"].shape)
top20 = torch.argsort(bert_out["prediction_logits"], dim=2, descending=True)[:,:,:20]
print(top20.shape)

print("Guesses:",tokenizer.decode(top20[0,4]))

torch.Size([1, 12, 28996])
torch.Size([1, 12, 20])
Guesses: have eat chase see pet keep play watch get scare be hunt ride like visit kill feed fight lick catch


In [ ]:
print(t)
print(tokenizer.mask_token_id)

{'input_ids': tensor([[  101, 16406,  1176,  1106,   103, 11771,   119,  1220,  1132, 10509,
           119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
103


# TASKS

As an exercise, you can try to solve the following:

1. How good is BERT at the masked language modelling (MLM) task? Feed random texts e.g. from the IMDB dataset, mask a random token at a time, and check: did BERT predict it correctly?
2. If you did (1), can you answer did BERT predict it correctly in top-5?
3. Try can you do better. Make yourself a program which picks random texts from one of the datasets we used in this course and produces two files: one with segments of texts with one [MASK] and one with the correct answers. Then try to guess the words without looking at the latter file and then compare your answers with the correct ones. How well did you do?


# General notes:

* Some had problems (or really bad reported performance) because of failing to select predictions for the correct token
* In the example code, the index of the masked token was hard-coded: **top20[0,4]**
* A typical error was to manually select the index based on the original text, not the tokenized text
  - _Dogs like to [MASK] cats. They are cute._ --> fourth token, so index=3
  - _[CLS] Dogs like to [MASK] cats . They are cute . [SEP]_ --> fifth token, so index=4
  - Note that tokenizer can also split tokens into subwords, so cannot assume just +1 for inserted CLS!
  - _[CLS] D ##O ##GS like to [MASK] cats . They are cute . [SEP]_ --> seventh token, so index=6




In [ ]:
print(dir(tokenizer))
text = "Dogs like to [MASK] cats. They are cute."
tok = tokenizer(texts[0])
print(tok)
detok = tokenizer.convert_ids_to_tokens(tok["input_ids"])
print(detok)
print(detok[4])

['SPECIAL_TOKENS_ATTRIBUTES', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_tokens', '_additional_special_tokens', '_auto_class', '_batch_encode_plus', '_bos_token', '_call_one', '_cls_token', '_convert_encoding', '_convert_id_to_token', '_convert_token_to_id_with_added_voc', '_create_repo', '_decode', '_decode_use_source_tokenizer', '_encode_plus', '_eos_token', '_eventual_warn_about_too_long_sequence', '_eventually_correct_t5_max_length', '_from_pretrained', '_get_files_timestamps', '_get_padding_truncation_strategies', '_in_target_context_manager', '_mask_token', '_pad', '_pad_token', '_pad_token_type_id', '_processor_class', '_save_pretrained

In [ ]:
text = "DOGS LIKE TO [MASK] CATS. THEY ARE CUTE."
tok = tokenizer(text)
print(tok)
detok = tokenizer.convert_ids_to_tokens(tok["input_ids"])
print(detok)
print(detok[8])

{'input_ids': [101, 141, 2346, 13472, 149, 2240, 22441, 16972, 103, 8784, 11365, 119, 7462, 3663, 22133, 2036, 140, 16830, 2036, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'D', '##O', '##GS', 'L', '##I', '##KE', 'TO', '[MASK]', 'CA', '##TS', '.', 'THE', '##Y', 'AR', '##E', 'C', '##UT', '##E', '.', '[SEP]']
[MASK]


# Genaralize to mask a random token and get predictions for it

In [ ]:
#Get some data
dataset = datasets.load_dataset("imdb", split="train") # we need only training section
dataset = dataset.shuffle()
dataset = dataset.select(range(100)) # downsample
print(dataset)
print(dataset[0])

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})
{'text': 'I rented this movie because it sounded pretty interesting but to my Horror this movie was the worst movie I had ever seen! I read the comment from Gumby-8 and he has to be a part of the cast or the crew. Unless Gumby-8 is a 4 year old child or some demented cult member no one in their right mind would think that this movie had any potential at all.<br /><br />I couldn\'t believe Gumby-8\'s comments. <br /><br />Quoting: "From the "Dune" inspired opening animation to the quick pace...this film keeps the eye moving and works so well that repeat viewing is not unexpected."<br /><br />The Animation is the only aspect of the movie that was interesting and the fact of that the film keeps the eye moving, well that\'s because you keep looking for any type of suspense. I mean give me a break Halloween was shot with a budget of $100,000.00 and a painted mask and also by the way became a cult classic. As far as "repeat view

In [ ]:
# truncate review texts to make it easier to spot things, this step is not necessary!
# Note that this can make the task easier / more difficult compared to full reviews

def truncate_review(example):
  text = example["text"]
  tokens = text.split() # naive tokenization, split on whitespace
  tokens = tokens[:30] # let's keep first 30 tokens
  short_text = " ".join(tokens) # convert back to string
  return {"text": short_text}

dataset = dataset.map(truncate_review)
print(dataset)
print(dataset[0])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 100
})
{'text': 'I rented this movie because it sounded pretty interesting but to my Horror this movie was the worst movie I had ever seen! I read the comment from Gumby-8 and', 'label': 0}


In [ ]:
# Let's test how masking of a random token works

import random

#help(tokenizer.decode)

text = dataset[0]["text"]

t = tokenizer(text) # prepare normal tokenized input
print("input_ids:", t["input_ids"])
print("tokens:", tokenizer.decode(t["input_ids"]))

random_idx = random.randint(1, len(t["input_ids"])-2) # sample random token, -2 so that we do not mask the last token which is [SEP]
print("random index:", random_idx, tokenizer.decode(t["input_ids"][random_idx]))

t["input_ids"][random_idx] = tokenizer.mask_token_id # mask the token in input
print("masked tokens:", tokenizer.decode(t["input_ids"]))




input_ids: [101, 146, 12765, 1142, 2523, 1272, 1122, 4234, 2785, 5426, 1133, 1106, 1139, 16709, 1142, 2523, 1108, 1103, 4997, 2523, 146, 1125, 1518, 1562, 106, 146, 2373, 1103, 7368, 1121, 144, 1818, 2665, 118, 129, 1105, 102]
tokens: [CLS] I rented this movie because it sounded pretty interesting but to my Horror this movie was the worst movie I had ever seen! I read the comment from Gumby - 8 and [SEP]
random index: 4 movie
masked tokens: [CLS] I rented this [MASK] because it sounded pretty interesting but to my Horror this movie was the worst movie I had ever seen! I read the comment from Gumby - 8 and [SEP]


In [ ]:
# define function to tokenize and mask
def tokenize_and_mask_random(text):
  t = tokenizer(text, return_tensors="pt")
  random_token_idx = random.randint(1, t["input_ids"].shape[1]-2) # sample random token
  original_token = tokenizer.decode(t["input_ids"][0, random_token_idx]) # get the original token
  t["input_ids"][0, random_token_idx] = tokenizer.mask_token_id # mask the token
  return t, random_token_idx, original_token

# define function to get predictions for masked token
def get_predictions(tokenized, random_token_idx):
  bert_out = bert(**tokenized)
  top5 = torch.argsort(bert_out["prediction_logits"], dim=2, descending=True)[:,:,:5] # lets take top5 to answer both 1 and 2
  pred_for_mask = top5[0, random_token_idx] # from first input text, select the predictions for masked token
  return [tokenizer.decode(t) for t in pred_for_mask]

# test with one example
tokenized, random_token_idx, original_token = tokenize_and_mask_random(dataset[0]["text"])
pred_for_mask = get_predictions(tokenized, random_token_idx)

print(dataset[0]["text"])
print(tokenizer.decode(tokenized["input_ids"][0,:]))
print("Original token:", original_token)
print("Predictions:", pred_for_mask)


I rented this movie because it sounded pretty interesting but to my Horror this movie was the worst movie I had ever seen! I read the comment from Gumby-8 and
[CLS] I rented this [MASK] because it sounded pretty interesting but to my Horror this movie was the worst movie I had ever seen! I read the comment from Gumby - 8 and [SEP]
Original token: movie
Predictions: ['movie', 'one', 'film', ',', 'thing']


In [ ]:
# run all
# Note: not optimized for speed!

top1_acc = 0
top5_acc = 0

for i,review in enumerate(dataset):
  tokenized, random_token_idx, original_token = tokenize_and_mask_random(review["text"])
  pred_for_mask = get_predictions(tokenized, random_token_idx)
  if pred_for_mask[0]==original_token:
    top1_acc+=1
  if original_token in pred_for_mask:
    top5_acc+=1
  if i < 5: # print first 5 examples for visualization
    print(i)
    print(tokenizer.decode(tokenized["input_ids"][0]))
    print(original_token)
    print(pred_for_mask)
    print()

print("Top-1 accuracy:", top1_acc/len(dataset)*100)
print("Top-5 accuracy:", top5_acc/len(dataset)*100)

0
[CLS] I rented this movie because it sounded pretty interesting but to my Horror this movie was the worst movie [MASK] had ever seen! I read the comment from Gumby - 8 and [SEP]
I
['I', 'anyone', 'he', 'we', 'they']

1
[CLS] Moonwalker is absolutely incredible!!!!!!! What [MASK] can I say!? Michael Jackson is the true King of pop, rock and soul!!! Moonwalker has everything! Great story line, [SEP]
else
['else', 'more', 'exactly', 'words', 'other']

2
[CLS] To Die For ( 1989 ) was just another d. t. v. feature that made an appearance on cable ad nasuem during the early nineties. The only thing notable [MASK] this feature was the [SEP]
about
['about', 'in', 'for', 'of', 'from']

3
[CLS] All Dogs Go To [MASK] is on a par with Watership Down for scary kiddies films. Both were dark and pretty sinister, but at the same time the most mesmerising [SEP]
Heaven
['Hell', 'School', 'Heaven', 'Sleep', 'War']

4
[CLS] I got to see this [MASK] at a preview and was dazzled by it. It's not the typica

In [ ]:
# data for human experiment

targets = []
for i, review in enumerate(dataset):
  tokenized, random_token_idx, original_token = tokenize_and_mask_random(review["text"])
  targets.append(original_token)
  print(i, tokenizer.decode(tokenized["input_ids"][0]))
  if i>=10: # skip rest
    break

0 [CLS] I rented this movie because it sounded pretty interesting but to my [MASK] this movie was the worst movie I had ever seen! I read the comment from Gumby - 8 and [SEP]
1 [CLS] Moonwalker is [MASK] incredible!!!!!!! What else can I say!? Michael Jackson is the true King of pop, rock and soul!!! Moonwalker has everything! Great story line, [SEP]
2 [CLS] To Die For ( 1989 ) was just another d. t. v. feature [MASK] made an appearance on cable ad nasuem during the early nineties. The only thing notable about this feature was the [SEP]
3 [CLS] All Dogs Go To Heaven is on a par [MASK] Watership Down for scary kiddies films. Both were dark and pretty sinister, but at the same time the most mesmerising [SEP]
4 [CLS] I got to see this film at a preview and was dazzled by it. It's not the typical [MASK] comedy. I can't remember laughing so hard at a film and [SEP]
5 [CLS] " Look, I know this may suck right now, but pain is temporary, film is forever. Whatever you do right now is burned int

In [ ]:
for i, label in enumerate(targets):
  print(i, label)

0 Horror
1 absolutely
2 that
3 with
4 romantic
5 cell
6 that
7 C
8 Ed
9 /
10 horror
